In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=0ff1acb5ab6b678b58d1920305b027151089cd921d31932b0cd5c16e89362e65
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 32 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package

In [9]:
import pyspark
import pyspark.sql  as pyspark_sql
import pyspark.sql.types as T
import pyspark.sql.functions  as F
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder.master('local[*]').getOrCreate()

In [7]:
print(spark.version)

3.5.0


In [8]:
actor_df = spark.read.csv('actor.csv', header=True, inferSchema=True)
address_df = spark.read.csv('address.csv', header=True, inferSchema=True)
category_df = spark.read.csv('category.csv', header=True, inferSchema=True)
city_df = spark.read.csv('city.csv', header=True, inferSchema=True)
country_df = spark.read.csv('country.csv', header=True, inferSchema=True)
customer_df = spark.read.csv('customer.csv', header=True, inferSchema=True)
film_df = spark.read.csv('film.csv', header=True, inferSchema=True)
film_actor_df = spark.read.csv('film_actor.csv', header=True, inferSchema=True)
film_category_df = spark.read.csv('film_category.csv', header=True, inferSchema=True)
inventory_df = spark.read.csv('inventory.csv', header=True, inferSchema=True)
language_df = spark.read.csv('language.csv', header=True, inferSchema=True)
payment_df = spark.read.csv('payment.csv', header=True, inferSchema=True)
rental_df = spark.read.csv('rental.csv', header=True, inferSchema=True)
staff_df = spark.read.csv('staff.csv', header=True, inferSchema=True)
store_df = spark.read.csv('store.csv', header=True, inferSchema=True)

Домашнє завдання на тему Spark SQL
Задачі з домашнього завдання на SQL потрібно розвʼязати за допомогою Spark SQL DataFrame API.

Дампи таблиць знаходяться в папці data. Датафрейми таблиць вже створені в клітинці вище.
Можете створювати стільки нових клітинок, скільки вам необхідно.
Розвʼязок кожної задачі має бути відображений в самому файлі (використати метод .show())
код має бути оформлений у відповідності із одним із стилем, показаним лектором на занятті 13.
Увага! Використовувати мову запитів SQL безпосередньо забороняється, потрібно використовувати виключно DataFrame API!


Вивести кількість фільмів в кожній категорії. Результат відсортувати за спаданням.

In [26]:
films_count_by_category = (film_df
                           .join(film_category_df, on='film_id')
                           .join(category_df, on='category_id')
                           .groupBy(category_df.name)
                           .count()
                           .sort(F.col("count").desc())
                           .show())


+-----------+-----+
|       name|count|
+-----------+-----+
|     Sports|   74|
|    Foreign|   73|
|     Family|   69|
|Documentary|   68|
|  Animation|   66|
|     Action|   64|
|        New|   63|
|      Drama|   62|
|      Games|   61|
|     Sci-Fi|   61|
|   Children|   60|
|     Comedy|   58|
|     Travel|   57|
|   Classics|   57|
|     Horror|   56|
|      Music|   51|
+-----------+-----+



Вивести 10 акторів, чиї фільми брали на прокат найбільше. Результат відсортувати за спаданням.

In [27]:
top_actors = (film_df
              .join(film_actor_df, on='film_id')
              .join(actor_df, on='actor_id')
              .join(inventory_df, on='film_id')
              .join(rental_df, on='inventory_id')
              .groupBy(actor_df.actor_id, actor_df.first_name, actor_df.last_name)
              .count()
              .sort(F.col("count").desc())
              .show(10))


+--------+----------+-----------+-----+
|actor_id|first_name|  last_name|count|
+--------+----------+-----------+-----+
|     107|      GINA|  DEGENERES|  753|
|     181|   MATTHEW|     CARREY|  678|
|     198|      MARY|     KEITEL|  674|
|     144|    ANGELA|WITHERSPOON|  654|
|     102|    WALTER|       TORN|  640|
|      60|     HENRY|      BERRY|  612|
|     150|     JAYNE|      NOLTE|  611|
|      37|       VAL|     BOLGER|  605|
|      23|    SANDRA|     KILMER|  604|
|      90|      SEAN|    GUINESS|  599|
+--------+----------+-----------+-----+
only showing top 10 rows




Вивести категорія фільмів, на яку було витрачено найбільше грошей в прокаті

In [31]:
most_expensive = (category_df
             .join(film_category_df, on='category_id')
             .join(film_df, on='film_id')
             .join(inventory_df, on='film_id')
             .join(rental_df, on='inventory_id')
             .join(payment_df, on='rental_id')
             .groupBy(F.col('name'))
             .agg(F.round(F.sum('amount'), 2).alias("total_spent"))
             .orderBy(F.col("total_spent").desc())
             .show(1))


+------+-----------+
|  name|total_spent|
+------+-----------+
|Sports|    5314.21|
+------+-----------+
only showing top 1 row



Вивести назви фільмів, яких не має в inventory.

In [32]:
films_not_in_inventory = (film_df
                          .join(inventory_df, film_df.film_id == inventory_df.film_id, "left_anti")
                          .select(film_df["title"])
                          .show())

+--------------------+
|               title|
+--------------------+
|      ALICE FANTASIA|
|         APOLLO TEEN|
|      ARGONAUTS TOWN|
|       ARK RIDGEMONT|
|ARSENIC INDEPENDENCE|
|   BOONDOCK BALLROOM|
|       BUTCH PANTHER|
|       CATCH AMISTAD|
| CHINATOWN GLADIATOR|
|      CHOCOLATE DUCK|
|COMMANDMENTS EXPRESS|
|    CROSSING DIVORCE|
|     CROWDS TELEMARK|
|    CRYSTAL BREAKING|
|          DAZED PUNK|
|DELIVERANCE MULHO...|
|   FIREHOUSE VIETNAM|
|       FLOATS GARDEN|
|FRANKENSTEIN STRA...|
|  GLADIATOR WESTWARD|
+--------------------+
only showing top 20 rows




Вивести топ 3 актори, які найбільше зʼявлялись в категорії фільмів “Children”

In [33]:
top_actors_in_children = (film_df
                          .join(film_category_df, on='film_id')
                          .join(category_df, on='category_id')
                          .where(category_df.name == "Children")
                          .join(film_actor_df, on='film_id')
                          .join(actor_df, 'actor_id')
                          .groupBy(actor_df.actor_id, actor_df.first_name, actor_df.last_name)
                          .count()
                          .orderBy(F.col("count").desc())
                          .show(3))

+--------+----------+---------+-----+
|actor_id|first_name|last_name|count|
+--------+----------+---------+-----+
|      17|     HELEN|   VOIGHT|    7|
|     127|     KEVIN|  GARLAND|    5|
|      80|     RALPH|     CRUZ|    5|
+--------+----------+---------+-----+
only showing top 3 rows



In [34]:
spark.stop()